# OscarNet


## Imports

In [ ]:
import keras
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras import layers
from keras import optimizers
from keras import regularizers
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import Model
from keras.layers import Dense, Activation,Reshape, Flatten, Conv2D, Dropout, BatchNormalization, Input, MaxPooling2D, Conv2DTranspose
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os, os.path
from keras.models import model_from_json
import sklearn
from sklearn.metrics import e, auc, roc_auc_score
import pandas as pd
import seaborn as sn
#from scipy.misc import imread
#from scipy.misc import imshow
import matplotlib
matplotlib.use("Agg")
import cv2

http://cs231n.github.io/transfer-learning/

## Data Processing

rotation_range is a value in degrees (0-180), a range within which to randomly rotate pictures

width_shift and height_shift are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally

rescale is a value by which we will multiply the data before any other processing. Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our models to process (given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255. factor.

shear_range is for randomly applying shearing transformations

zoom_range is for randomly zooming inside pictures

horizontal_flip is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).

fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        #zca_epsilon=True,
        #featurewise_center = True,
        #zca_whitening=True, # apply ZCA whitening
        rotation_range=20, # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
        horizontal_flip=True, # randomly flip images
        vertical_flip=False) # randomly flip images


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'dataset_resized/train',  
        target_size=(224, 224),  
        batch_size=batch_size,
        class_mode='categorical')  

validation_generator = test_datagen.flow_from_directory(
        'dataset_resized/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')



In [ ]:
len_dataset = 2258
itr = train_datagen.flow_from_directory(
        'dataset_resized/train',  
        target_size=(224, 224),  
        batch_size=len_dataset,
        class_mode='categorical')  

X, y = itr.next()

## ImageNet Networks

In [ ]:
def load_vgg_model(filename):
    json_file = open(filename, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.layers.pop()
    loaded_model.layers.pop()
    loaded_model.layers.pop()
    return_model = Sequential()
    for layer in loaded_model.layers:
        return_model.add(layer)
    return return_model

In [ ]:
def save_model(filename, model):
    model_json = model.to_json()
    with open(filename, "w") as json_file:
        json_file.write(model_json)


In [ ]:
vgg_model = keras.applications.vgg19.VGG19(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)

In [ ]:
resnet50_model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)

In [ ]:
incep_model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)

In [ ]:
alt_model = vgg_model

In [ ]:
alt_model.layers.pop()
alt_model.layers.pop()
alt_model.layers.pop()
#alt_model.summary()

In [ ]:
#alt_model.summary()
save_model("models/vgg.json", vgg_model)

In [ ]:
num_layers_retrain = len(alt_model.layers)
num_layers_retrain

In [ ]:
for layer in alt_model.layers[:num_layers_retrain]:
    layer.trainable = False
    # how many FC layers and what size 
# what activations should we use
# what is the output size
# 
output = alt_model.layers[len(alt_model.layers)-1].output_shape
output                

## Transfer Network Architectures

### First Network

In [ ]:
num_classes = 7

transfer_net_test = Sequential()
transfer_net_test.add(Dense(256, input_shape = output,activation='relu'))
transfer_net_test.add(Dense(num_classes, activation='softmax'))

In [ ]:
vgg_test = load_vgg_model('models/vgg.json')
vgg_test.add(transfer_net_test)

In [ ]:
adam = keras.optimizers.Adam(lr=.001, beta_1=0.9, beta_2=0.999, epsilon=.1, decay=0.0001)

vgg_test.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['categorical_accuracy'])

vgg_test.fit_generator(
        train_generator,
        steps_per_epoch=100, 
        epochs=10,
        validation_data=validation_generator,
        validation_steps=15)

#### Lessons:
    - Try BatchNormalization
    - Try Dropout
    - Try increasing layer size
    - Try increasing network depth

Network 1

In [ ]:
def transfer_network(params, output): 
    transfer_net = Sequential()
    transfer_net.add(Dense(params['first_layer'], input_shape = output,activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params['reg']),))
    transfer_net.add(Dropout(params['dropout']))
    transfer_net.add(BatchNormalization())
    for layer in range(0,params['extra_layers']):
            transfer_net.add(Dense(params['first_layer'] / 2, input_shape = output,activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params['reg']),))
            transfer_net.add(Dropout(params['dropout']))
            transfer_net.add(BatchNormalization())
                                  
    transfer_net.add(Dense(7, activation='softmax', kernel_initializer='zeros', bias_initializer='zeros'))
    return transfer_net

# Training Transfer Network

In [ ]:
def train_model(x, generator, params):

    adam = keras.optimizers.Adam(lr=params['lr'], beta_1=0.9, beta_2=0.999, epsilon=params['epsilon'], decay=0.0001)

    x.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['categorical_accuracy'])

    return x.fit_generator(
            generator,
            steps_per_epoch=params['steps'], 
            epochs=params['epochs'],
            validation_data=validation_generator,
            validation_steps=15)

In [ ]:
params = {
    'lr': .005,
    'epsilon': .1,
    'epochs' : 10,
}


# Hyperparameter Search

In [ ]:
def load_generator(batch_size, datagen):
    return datagen.flow_from_directory(
            'dataset_resized/train', 
            target_size=(224, 224),  
            batch_size=batch_size,
            class_mode='categorical')
        

In [ ]:
param = {
     'lrs': [.0005, .001, .005, .01, .1],
     'layer_size':[64, 128, 256],
     'batch_size': [16, 24, 32, 64],
     'epochs': [5],
     'dropout': [.2, .4, .6, .8],
     'epsilons': [.1, .25, .5, .75, 1],
     'losses': ['categorical_crossentropy'],
     'regularizer': [.01, .02, .05, .1, .5],
  
 }


for size in param['layer_size']:
    for d in param['dropout']:
        for reg in param['regularizer']:
            x = load_vgg_model('models/vgg.json')
            net_params = {
                'first_layer': 256,
                'reg': reg,
                'dropout': d,
                'extra_layers' : 0
            }
            x.add(transfer_network(net_params, output))
            Wsave = x.get_weights()
            for batch_size in params['batch_sizes']:
                train_generator = load_generator(batch_size, train_datagen) 
                for lr in params['lrs']:
                    for epsilon in params['epsilons']:
                        print("-----------------------")
                        print(" ")
                        print(batch_size, lr, epsilon)
                        params = {
                            'lr': lr,
                            'epsilon': epsilon,
                            'steps': 30,
                            'epochs' : 10,
                        }
                        x.set_weights(Wsave)
                        output = train_model(x, train_generator, params)
                        print("-----------------------")
                        print(" ")
                        outputs.append(output)          

In [ ]:
for output in outputs:
    print(output.history["categorical_accuracy"], output.history["loss"], output.history["val_categorical_accuracy"], output.history["val+loss"])

## What did we learn?

A larger mini-batch leads to better, faster results (in terms of epochs). However, the smaller batches ran quicker. Across all batches, we saw loss consistently decrease and accuracy consistently increase as the epoch increased, so we decided to balance speed of training with a larger batch size and chose a mini-batch size of 32.

For our lr, we saw that a higher learning rate led to faster, but more variable training. We fonud that learning rates between .001 and .0001 returned good results. We decided to use different learning rates based on what stage of training we were at, as a way to control the rate of training. 

For our epsilon, we saw that 0 returned nan for loss, and was not a viable option. Upon investgation, we found .5 to be the best parameter for this.

# Plotting Training Results

In [ ]:
plt.style.use("ggplot")
def plot_curves(outputs, N, name, filename):
    plt.figure()
    plt.plot(np.arange(0, N), outputs.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), outputs.history["val_loss"], label="val_loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.title(name)
    plt.legend(loc="lower left")
    plt.savefig(filename+"loss")
    plt.figure()
    plt.plot(np.arange(0, N), outputs.history["categorical_accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), outputs.history["val_categorical_accuracy"], label="val_acc")
    plt.title(name)
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(filename+"acc")

In [ ]:
def save_stats(output, name):
    loss_history = output.history["loss"]
    val_loss_history = output.history["val_loss"]
    acc_history = output.history["categorical_accuracy"]
    val_acc_history = output.history["val_categorical_accuracy"]
    numpy_loss_history = np.array(loss_history)
    np.savetxt("loss_history" + name + ".txt", numpy_loss_history, delimiter=",")
    numpy_val_loss_history = np.array(val_loss_history)
    np.savetxt("val_loss_history" + name + ".txt", numpy_val_loss_history, delimiter=",")
    numpy_acc_history = np.array(acc_history)
    np.savetxt("acc_history" + name + ".txt", numpy_acc_history, delimiter=",")
    numpy_val_acc_history = np.array(val_acc_history)
    np.savetxt("val_acc_history" + name + ".txt", numpy_val_acc_history, delimiter=",")
    trained_model.save_weights('weights_shorttrain1.h5')


## Short Train

In [ ]:
x = load_vgg_model('models/vgg.json')

for l in x.layers:
    l.trainable = False

output = x.layers[len(x.layers)-1].output_shape

net_params = {
    'first_layer': 256,
    'reg': 0,
    'dropout': .6,
    'extra_layers' : 0
}

params = {
    'lr': .0001,
    'epsilon': .1,
    'steps': len_dataset/32,
    'epochs' : 10,
}

x.add(transfer_network(net_params, output))

short_output = train_model(x, train_generator, params  )

save_stats(short_output, "shorttrain")

x.save_weights('weights/weights_shorttrain_updated.h5')

plot_curves(short_output, params['epoch'], "Short Training with learning rate .005", "plots/shorttrain")

## Medium Train

In [ ]:
x = load_vgg_model('models/vgg.json')

for l in x.layers:
    l.trainable = False

for l in x.layers[len(x.layers)-3:]:
    l.trainable = True

output = x.layers[len(x.layers)-1].output_shape

net_params = {
    'first_layer': 256,
    'reg': .01,
    'dropout': .6,
    'extra_layers' : 0
}

params = {
    'lr': .005,
    'epsilon': .5,
    'steps': len_dataset/32,
    'epochs' : 50,
}

x.add(transfer_network(net_params, output))

x.load_weights('weights/weights_shorttrain_updated.h5')

med_output = train_model(x, train_generator, params  )

save_stats(med_output, "shorttrain")

x.save_weights('weights/weights_medtrain_updated.h5')

plot_curves(med_output, params['epoch'], "Short Training with learning rate .005", "plots/shorttrain")

## Long Train

In [ ]:
x = load_vgg_model('models/vgg.json')

for l in x.layers:
    l.trainable = False

for l in x.layers[len(x.layers)-6:]:    
    l.trainable = True

output = x.layers[len(x.layers)-1].output_shape

net_params = {
    'first_layer': 256,
    'reg': .025,
    'dropout': .6,
    'extra_layers' : 0
}

params = {
    'lr': .0001,
    'epsilon': .1,
    'steps': len_dataset/32,
    'epochs' : 200,
}

x.add(transfer_network(net_params, output))

x.load_weights('weights_longtrain2.h5')

long_output = train_model(x, train_generator, params  )

save_stats(long_output, "shorttrain")

x.save_weights('weights_longtrain3.h5')

plot_curves(long_output, params['epoch'], "Short Training with learning rate .005", "plots/shorttrain")

In [ ]:
x.save_weights('weights_longtrain2.h5')

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

x = load_vgg_model('models/vgg.json')


net_params = {
    'first_layer': 256,
    'reg': .05,
    'dropout': .6,
    'extra_layers' : 0
}

output = x.layers[len(x.layers)-1].output_shape

x.add(transfer_network(net_params, output))

x.load_weights('weights_longtrain1.h5')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16

validation_generator = test_datagen.flow_from_directory(
        'dataset_resized/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

y_true = []
for name in validation_generator.filenames:
    if ("trash" in name): y_true.append(6)
    if ("metal" in name): y_true.append(2)
    if ("plastic" in name): y_true.append(5)
    if ("glass" in name): y_true.append(1)
    if ("paper" in name): y_true.append(4)
    if ("waste" in name): y_true.append(3)
    if ("cardboard" in name): y_true.append(0)
y_true = np.array(y_true)

In [ ]:
probabilities = x.predict_generator(validation_generator)


y_pred = np.array([(np.argmax(l)) for l in probabilities])

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


In [ ]:
validation_generator.class_indices.keys()

In [ ]:
q = confusion_matrix(y_true, y_pred)
_max, _min = q.max(), q.min()
q = (q - _min)/(_max - _min)
q

In [ ]:
data_frame = pd.DataFrame(q, index = [i for i in validation_generator.class_indices.keys()],
                  columns = [i for i in validation_generator.class_indices.keys()])
sn.heatmap(data_frame, annot=True)

In [ ]:
r = auc(y_true, y_pred)